# Dev

In [ ]:
from sae_training.config import LanguageModelSAERunnerConfig
from sae_training.utils import LMSparseAutoencoderSessionloader

import torch 
torch.cuda.empty_cache()

cfg = LanguageModelSAERunnerConfig(
    n_batches_in_buffer=25,
    store_batch_size=512,
    context_size=128)

path ="artifacts/sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768:v7/final_sparse_autoencoder_gelu-2l_blocks.0.hook_mlp_out_32768.pt"
model, sparse_autoencoder, activations_loader = LMSparseAutoencoderSessionloader.load_session_from_pretrained(
    path
)


In [ ]:
activation_batch = activations_loader.next_batch()
activation_batch.shape

In [ ]:
sae_out, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(activation_batch)
print(sae_out.shape)
print(feature_acts.shape)
print(loss)
print(mse_loss)
print(l1_loss)

In [ ]:
(feature_acts > 0)

In [ ]:
l0 = (feature_acts > 0).float().sum(0).mean()
l0

In [ ]:
(feature_acts > 0).float().sum(0).shape

In [ ]:
(feature_acts > 0).float().sum(1).mean()

In [ ]:
feature_acts.shape

In [ ]:
from sae_training.train_sae_on_language_model import get_recons_loss

score, loss, recons_loss, zero_abl_loss = get_recons_loss(
    sparse_autoencoder, model, activations_loader, num_batches=10,
)

In [ ]:
recons_loss

In [ ]:
loss

In [ ]:
zero_abl_loss

In [ ]:
score

In [ ]:
batch = activations_loader.ge

In [ ]:

from functools import partial
batch_tokens = activations_loader.get_batch_tokens()


def replacement_hook(mlp_post, hook, encoder):
    mlp_post_reconstr = encoder(mlp_post)[0]
    return mlp_post_reconstr


def mean_ablate_hook(mlp_post, hook):
    mlp_post[:] = mlp_post.mean([0, 1])
    return mlp_post


def zero_ablate_hook(mlp_post, hook):
    mlp_post[:] = 0.0
    return mlp_post

hook_point= sparse_autoencoder.cfg.hook_point

logits = model.run_with_hooks(
            batch_tokens,
            fwd_hooks=[(hook_point, partial(replacement_hook, encoder=sparse_autoencoder))],
        )

per_token_loss = model.loss_fn(logits, batch_tokens, True).detach().cpu().numpy()

In [ ]:
logits = model(batch_tokens)

per_token_loss_original = model.loss_fn(logits, batch_tokens, True).detach().cpu().numpy()

In [ ]:
from importlib import reload
from plotly import express as px
reload(px)


fig = px.histogram(per_token_loss.flatten())
fig.write_image("image.png")

In [ ]:
model.tokenizer.decode(batch_tokens[0])

In [ ]:
logits, cache = model.run_with_cache(batch_tokens)

In [ ]:
logits.shape

In [ ]:
cache[hook_point]

In [ ]:
_, feature_acts, loss, mse_loss, l1_loss = sparse_autoencoder(cache[hook_point])

In [ ]:
feature_acts.shape

In [ ]:

fig = px.imshow(per_token_loss, color_continuous_midpoint=0, color_continuous_scale="RdBu")
fig.write_image("image1.png")

In [ ]:

fig = px.imshow((feature_acts > 0)[:,1:].sum(-1).detach().cpu().numpy(), color_continuous_midpoint=0, color_continuous_scale="RdBu")
fig

batch_tokens.shape

In [ ]:
model.tokenizer.batch_decode(batch_tokens[:,1:].flatten())[0]


In [ ]:
fig = px.scatter(
    x = (feature_acts > 0)[:,1:].sum(-1).flatten().detach().cpu().numpy(),
    y = per_token_loss_original.flatten() - per_token_loss.flatten(),
    marginal_x="histogram",
    marginal_y="histogram",
    labels = {
        "x": "Number of features that fired", "y": "Per-token loss"
    },
    hover_data=[model.tokenizer.batch_decode(batch_tokens[:,1:].flatten())],
    color= torch.tensor([range(1,128) for i in range(32)]).flatten(),
    log_x=True,
    log_y=True,
    opacity=0.5,
)
fig

In [ ]:
fig = px.scatter(
    x = (feature_acts > 0)[:,1:].sum(-1).flatten().detach().cpu().numpy(),
    y = per_token_loss.flatten(),
    marginal_x="histogram",
    marginal_y="histogram",
    labels = {
        "x": "Number of features that fired", "y": "Per-token loss"
    },
    hover_data=[model.tokenizer.batch_decode(batch_tokens[:,1:].flatten())],
    color= torch.tensor([range(1,128) for i in range(32)]).flatten(),
    log_x=True,
    opacity=0.5,
)
fig


In [ ]:
torch.tensor([range(1,128) for i in range(32)]).flatten()

In [ ]:
model.tokenizer.batch_decode(batch_tokens.flatten())